In [1]:
import pickle
import pandas as pd

In [2]:
df = pd.read_csv('D:/Projects/ML/Personalized Recommendation System/dataset/ratings_Electronics (1).csv',names= ['userId', 'productId', 'Rating', 'timestamp'])

In [3]:
df['Rating'] = df['Rating'].astype(int)
df['userId'] = df['userId'].astype('str')
df['productId'] = df['productId'].astype('str')

df = df.drop(columns=['timestamp'])

In [4]:
filename= 'model/recommendation_model_svd.pkl'
loaded_model= pickle.load(open(filename, 'rb'))

In [9]:
# users ex. A25C2M3QF9G7OQ , A6FIAB28IS79

user_id = 'A25C2M3QF9G7OQ' 

items = df['productId'].unique()

rated_items = df[df['userId'] == user_id]['productId'].values
unrated_items = [item for item in items if item not in rated_items]

predictions = [loaded_model.predict(user_id, item) for item in unrated_items]

sorted_predictions = sorted(predictions, key=lambda x: x.est, reverse=True)

# Print the predicted ratings
for prediction in sorted_predictions[:10]:
    print(f"Item {prediction.iid} - Predicted Rating: {prediction.est}")

Item B0043WJRRS - Predicted Rating: 4.874813310284325
Item B007R5YGO2 - Predicted Rating: 4.854702224231724
Item B003FVVMS0 - Predicted Rating: 4.8231185502942635
Item B009NB8WR0 - Predicted Rating: 4.80289962664856
Item B000OGX5AM - Predicted Rating: 4.802638252871689
Item B00D5Q75RC - Predicted Rating: 4.797449216580759
Item B004LSNF04 - Predicted Rating: 4.7973104673152855
Item B0048IW030 - Predicted Rating: 4.79324842016439
Item B009WU5XUG - Predicted Rating: 4.790176355727978
Item B003XM1WE0 - Predicted Rating: 4.787511448568874
